Nous avons pris 2 exemples de textes récupéré de wikipedia pour les deux monuments : Mémorial de la Shoah et Mémorial des Martyrs de la Déportation


In [1]:
!pip install pyspotlight
!pip install SPARQLWrapper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import spotlight
import json
from SPARQLWrapper import SPARQLWrapper, JSON

## Fonction d'annotation

In [3]:
"""Annotation function"""
def spotlight_annotate(text, confidence=0.4, support=20):
    # Set the API endpoint URL
    endpoint = 'https://api.dbpedia-spotlight.org/en/annotate'

    # Set the parameters : text, confidence and support
    # Support : il s'agit du paramètre « Resource Prominence », qui aide à ignorer les ressources 
    # peu importantes ou peu informatives. Lorsque on lui attribue une valeur X, cela signifie que les
    # ressources dont le nombre de "in-links" Wikipedia est inférieur à X seront ignorées et ne 
    # seront pas renvoyées. 
    # Confidence : c'est le paramètre « Disambiguation Confidence », c'est un 
    # seuil qui prend une valeur entre 0 et 1. Lorsque on lui donne une valeur élevée, on obtient
    # des annotations meilleures et plus fiables mais on risque de perdre certaines annotations correctes

    # Send the request to the API endpoint
    response = spotlight.annotate(endpoint, text, confidence=confidence, support=support)

    return response
    

## Fonction pour stocker des annotations dans un fichier json


In [4]:
def store_annotations(data):
    # Convert the data to JSON 
    json_data = json.dumps(data, indent=4, separators=(',', ': '), sort_keys=True)
    # Store it in a file
    with open("annotated_data.json", "w") as outfile:
        outfile.write(json_data)

In [6]:
# Extracting the URIs from the JSON data. The function takes a list of dictionaries (data) as an input, 
# and for each dictionary in the list (annotation), it appends the value of the URI key to the URIs list.
def extract_URIs(data):
    URIs = []
    for annotation in data:
        URIs.append(annotation['URI'])
    return URIs

In [7]:
# Get a list of tuple containing URI and surfaceform for each annotation
def extract_URIs_entities(data):
    return [(annotation['URI'], annotation['surfaceForm']) for annotation in data]

### Lire les deux fichiers exemple

In [60]:
from google.colab import files
import codecs

# Upload the file to Google Colab
uploaded = files.upload()

with open("Mémorial de la Shoah.txt", "r", encoding="utf-8") as f:
  # Read the contents of the file
  text1 = f.read()


Saving Mémorial de la Shoah.txt to Mémorial de la Shoah (3).txt


In [61]:
print(text1)

Mémorial de la Shoah

Historique du mémorial

En avril 1943, pendant la Seconde Guerre mondiale à Grenoble, Isaac Schneersohn et Léon Poliakov fondèrent clandestinement le Centre de documentation juive contemporaine, dans le but de réunir des preuves documentaires sur la destruction des Juifs d'Europe.

En 1957, fut inauguré le Mémorial du Martyr juif inconnu. En janvier 2005, l'ensemble du site avec le Mur des Noms prend le nom de Mémorial de la Shoah.

Le 21 septembre 2012, un mémorial situé sur le site de l’ancien camp de Drancy1 a été inauguré par François Hollande, président de la République française2.

Le mémorial est présidé depuis 2005 par Éric de Rothschild.


In [36]:
response1 = spotlight_annotate(text1)
# Call the function to store the annotations in a json file
store_annotations(response1)
response1 = [entry for entry in response1 if entry['similarityScore'] >= 0.99]

# Print the annotations
print(response1)

[{'URI': 'http://dbpedia.org/resource/Mémorial_de_la_Shoah', 'support': 30, 'types': 'Wikidata:Q41176,Wikidata:Q33506,Schema:Place,DBpedia:Place,DBpedia:Location,DBpedia:Building,DBpedia:ArchitecturalStructure,DBpedia:Museum', 'surfaceForm': 'Mémorial de la Shoah', 'offset': 0, 'similarityScore': 1.0, 'percentageOfSecondRank': 0.0}, {'URI': 'http://dbpedia.org/resource/Pendant', 'support': 378, 'types': '', 'surfaceForm': 'pendant', 'offset': 61, 'similarityScore': 0.9999999970814315, 'percentageOfSecondRank': 2.9087849991348443e-09}, {'URI': 'http://dbpedia.org/resource/Grenoble', 'support': 4077, 'types': 'Wikidata:Q486972,Schema:Place,DBpedia:PopulatedPlace,DBpedia:Place,DBpedia:Location,DBpedia:Settlement', 'surfaceForm': 'Grenoble', 'offset': 98, 'similarityScore': 1.0, 'percentageOfSecondRank': 1.2765441920642218e-16}, {'URI': 'http://dbpedia.org/resource/Léon_Poliakov', 'support': 89, 'types': '', 'surfaceForm': 'Léon Poliakov', 'offset': 129, 'similarityScore': 0.99999997110035

In [42]:
def check_person(uris):
    # Initialize the SPARQLWrapper
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    
    valid_uris = []
    for URI in uris:
        # Define the query to get the rdf:type property
        query = f"SELECT ?type WHERE {{<{URI}> a ?type}}"
        # Set the query
        sparql.setQuery(query)
        # Set the return format as JSON
        sparql.setReturnFormat(JSON)
        # Execute the query
        results = sparql.query().convert()
        # Extract the rdf:type values
        types = [result["type"]["value"] for result in results["results"]["bindings"]]
        # Check if dbo:Person is in the rdf:type values
        if "http://dbpedia.org/ontology/Person" in types:
            valid_uris.append(URI)
            
    return valid_uris


In [43]:
uris1 = extract_URIs(response1)
uris1

['http://dbpedia.org/resource/Mémorial_de_la_Shoah',
 'http://dbpedia.org/resource/Pendant',
 'http://dbpedia.org/resource/Grenoble',
 'http://dbpedia.org/resource/Léon_Poliakov',
 'http://dbpedia.org/resource/Mur_(river)',
 'http://dbpedia.org/resource/Mémorial_de_la_Shoah',
 'http://dbpedia.org/resource/François_Hollande']

In [44]:
uris1 = check_person(uris1)
uris1

['http://dbpedia.org/resource/Léon_Poliakov',
 'http://dbpedia.org/resource/François_Hollande']

In [49]:
def get_historical_people_info(URIs):
    # Initialize the SPARQLWrapper
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")

    person_info = {}
    for URI in URIs:
        # Define the query to get the dbo:birthName and rdfs:comment property in French
        query = f"SELECT ?label ?abstract WHERE {{<{URI}> a dbo:Person . <{URI}> rdfs:label ?label . <{URI}> dbo:abstract ?abstract . FILTER (lang(?label) = 'fr') FILTER (lang(?abstract) = 'fr')}}"
        # Set the query
        sparql.setQuery(query)
        # Set the return format as JSON
        sparql.setReturnFormat(JSON)
        # Execute the query
        results = sparql.query().convert()
        # Extract the label and abstract values
        try:
            label = results["results"]["bindings"][0]["label"]["value"]
            abstract = results["results"]["bindings"][0]["abstract"]["value"]
        except:
            label = None
            abstract = None
        # Add the label and abstract to the person_info dictionary
        person_info[URI] = {}
        person_info[URI]["label"] = label
        person_info[URI]["abstract"] = abstract
        
    return person_info


In [50]:
info_dict = get_historical_people_info(uris1)
info_dict

{'http://dbpedia.org/resource/Léon_Poliakov': {'label': 'Léon Poliakov',
  'abstract': "Léon Poliakov (en russe : Лев Влади́мирович Поляко́в), né le 25 novembre 1910 à Saint-Pétersbourg et mort à Orsay le 8 décembre 1997, est un historien français. Il est un pionnier de l'histoire de la Shoah et un spécialiste de l'étude de l'antisémitisme."},
 'http://dbpedia.org/resource/François_Hollande': {'label': 'François Hollande',
  'abstract': "François Hollande [fʁɑ̃swa ʔɔlɑ̃d] , né le 12 août 1954 à Rouen, est un haut fonctionnaire et homme d'État français. Il est président de la République française du 15 mai 2012 au 14 mai 2017. Magistrat à la Cour des comptes et brièvement avocat, il est élu pour la première fois député en 1988. Il exerce la fonction de premier secrétaire du Parti socialiste (PS) de 1997 à 2008, pendant la troisième cohabitation puis dans l'opposition. Au niveau local, il est maire de Tulle de 2001 à 2008 et président du conseil général de Corrèze de 2008 à 2012. Désigné

In [62]:
uploaded = files.upload()

with open("Mémorial des Martyrs de la Déportation.txt", "r", encoding="utf-8") as f:
  # Read the contents of the file
  text2 = f.read()

Saving Mémorial des Martyrs de la Déportation.txt to Mémorial des Martyrs de la Déportation (4).txt


In [65]:
print(text2)

Mémorial des Martyrs de la Déportation
Situation et historique

Entrée du mémorial dans le square de l'Île-de-France.
Situé dans le 4e arrondissement de Paris, à la pointe est de l’île de la Cité en contrebas du square de l'Île-de-France, le mémorial a été commandé à l'initiative du Réseau du Souvenir qui en a fait don à l'État le 29 février 1964.

Le projet, réalisé par l'architecte Georges-Henri Pingusson, a été inauguré le 12 avril 1962 par le général de Gaulle, président de la République.


In [67]:
response2 = spotlight_annotate(text2)
store_annotations(response2)
response2 = [entry for entry in response2 if entry['similarityScore'] >= 0.99]
response2


[{'URI': 'http://dbpedia.org/resource/Entrée',
  'support': 142,
  'types': '',
  'surfaceForm': 'Entrée',
  'offset': 64,
  'similarityScore': 0.9930953961513407,
  'percentageOfSecondRank': 0.0},
 {'URI': 'http://dbpedia.org/resource/Arrondissements_of_Paris',
  'support': 172,
  'types': 'Wikidata:Q486972,Schema:Place,DBpedia:PopulatedPlace,DBpedia:Place,DBpedia:Location,DBpedia:Settlement',
  'surfaceForm': 'arrondissement',
  'offset': 135,
  'similarityScore': 0.9995417621448184,
  'percentageOfSecondRank': 0.0002863626581226001},
 {'URI': 'http://dbpedia.org/resource/Paris',
  'support': 136571,
  'types': 'Wikidata:Q515,Wikidata:Q486972,Schema:Place,Schema:City,DBpedia:Settlement,DBpedia:PopulatedPlace,DBpedia:Place,DBpedia:Location,DBpedia:City',
  'surfaceForm': 'Paris',
  'offset': 153,
  'similarityScore': 1.0,
  'percentageOfSecondRank': 2.4773912997544526e-19},
 {'URI': 'http://dbpedia.org/resource/Charles_de_Gaulle',
  'support': 3571,
  'types': 'Http://xmlns.com/foaf/0

In [68]:
uris2 = extract_URIs(response2)
uris2

['http://dbpedia.org/resource/Entrée',
 'http://dbpedia.org/resource/Arrondissements_of_Paris',
 'http://dbpedia.org/resource/Paris',
 'http://dbpedia.org/resource/Charles_de_Gaulle']

In [69]:
uris2 = check_person(uris2)
uris2

['http://dbpedia.org/resource/Charles_de_Gaulle']

In [71]:
info_dict2 = get_historical_people_info(uris2)
info_dict2

{'http://dbpedia.org/resource/Charles_de_Gaulle': {'label': 'Charles de Gaulle',
  'abstract': "Charles de Gaulle (/ʃaʁl də ɡol/ ), communément appelé le général de Gaulle ou parfois simplement le Général, né le 22 novembre 1890 à Lille et mort le 9 novembre 1970 à Colombey-les-Deux-Églises, est un militaire, résistant, homme d'État et écrivain français. Il est notamment chef de la France libre puis dirigeant du Comité français de libération nationale pendant la Seconde Guerre mondiale, président du Gouvernement provisoire de la République française de 1944 à 1946, président du Conseil des ministres de 1958 à 1959, instigateur de la Cinquième République, fondée en 1958, et président de la République de 1959 à 1969, étant le premier à occuper la magistrature suprême sous ce régime. Élevé dans une culture de grandeur nationale, Charles de Gaulle choisit une carrière d'officier. Au cours de la Première Guerre mondiale, il est blessé et fait prisonnier. Par la suite, il sert et publie dans

Nous pouvons enrichir notre ontologie historicalTrip en utilisant les données de Wikipedia et DBpedia. En parseant la section "histoire" de Wikipedia, nous pouvons extraire les informations sur les personnes associées à un site historique et utiliser DBpedia Spotlight pour les annoter. Nous pouvons alors ajouter ces triplets à notre ontologie pour la classe historicalSite en utilisant la propriété hasRelatedPerson. De plus, nous pouvons ajouter le nom et l'abstract de ces personnes pour plus de détails.